In [2]:
import os
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
import PIL.Image as Image

from skimage import io, color, transform
from skimage.feature import hog

import time

C:\Users\gonza\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## Carga de la base (unpickle the pickle)

In [3]:
images = np.load('images.npy', allow_pickle=True)
labels = np.load('labels.npy', allow_pickle=True)

df = pd.DataFrame({'gray_matrix': images.tolist(), 'label': labels.tolist()})
df

,gray_matrix,label
0,"[[46, 34, 26, 23, 12, 32, 35, 26, 26, 38, 71, ...",angry
1,"[[55, 33, 54, 32, 26, 21, 50, 29, 38, 45, 79, ...",angry
2,"[[123, 123, 126, 131, 124, 69, 109, 149, 159, ...",angry
3,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",angry
4,"[[127, 121, 124, 137, 123, 118, 120, 111, 111,...",angry
...,...,...
35882,"[[27, 26, 26, 26, 27, 27, 28, 28, 33, 34, 35, ...",surprise
35883,"[[7, 6, 13, 39, 76, 58, 29, 21, 16, 20, 34, 56...",surprise
35884,"[[124, 129, 125, 123, 129, 128, 127, 128, 131,...",surprise
35885,"[[242, 242, 242, 216, 110, 42, 20, 41, 37, 14,...",surprise


### Añadimos las caracteristicas - HOG

In [5]:
def calculate_hog(image):
    # Reshape de la imagen al tamaño requerido por HOG
    image_resized = cv2.resize(image, (64, 128))

    # Crear un objeto HOG
    hog = cv2.HOGDescriptor()

    # Calcular el HOG en la imagen redimensionada
    hog_features = hog.compute(image_resized)
    return hog_features

In [6]:
%%time

df['hog'] = df['gray_matrix'].apply(calculate_hog)
df

CPU times: total: 4.89 s
Wall time: 5.05 s


,gray_matrix,label,hog
0,"[[46, 34, 26, 23, 12, 32, 35, 26, 26, 38, 71, ...",angry,"[0.33274356, 0.1704117, 0.025357863, 0.0006549..."
1,"[[55, 33, 54, 32, 26, 21, 50, 29, 38, 45, 79, ...",angry,"[0.27381572, 0.22986299, 0.09233569, 0.0294371..."
2,"[[123, 123, 126, 131, 124, 69, 109, 149, 159, ...",angry,"[0.34384748, 0.01883176, 0.004236705, 0.010840..."
3,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",angry,"[0.014489936, 0.008741817, 0.020611065, 0.0, 0..."
4,"[[127, 121, 124, 137, 123, 118, 120, 111, 111,...",angry,"[0.25325525, 0.0767658, 0.026558537, 0.0181144..."
...,...,...,...
35882,"[[27, 26, 26, 26, 27, 27, 28, 28, 33, 34, 35, ...",surprise,"[0.14022486, 0.07403437, 0.0686184, 0.0, 0.041..."
35883,"[[7, 6, 13, 39, 76, 58, 29, 21, 16, 20, 34, 56...",surprise,"[0.39263126, 0.11640715, 0.0072888294, 0.00144..."
35884,"[[124, 129, 125, 123, 129, 128, 127, 128, 131,...",surprise,"[0.20560372, 0.04748373, 0.03934206, 0.0998374..."
35885,"[[242, 242, 242, 216, 110, 42, 20, 41, 37, 14,...",surprise,"[0.3589315, 0.16316827, 0.042510495, 0.0318309..."


# Undersampleo

In [7]:
from sklearn.neighbors import NearestNeighbors
def knn_und(X,y,majority_class, k, t, algorithm="auto", metric="euclidean"):
    #TODO IMPLEMENT SUPPORT FOR SPARSE MATRICES
    if not isinstance(X, (np.ndarray, np.generic) ):
        X = np.asarray(X)

    if not isinstance(y, (np.ndarray, np.generic) ):
        y = np.asarray(y)



    nrows = X.shape[0]
    ncols=1

    result= [True] * nrows

    nbrs = NearestNeighbors(n_neighbors=k,algorithm="auto", metric=metric).fit(X)
    nns_indexes = nbrs.kneighbors(X,return_distance=False)

    for i in range(nrows):
        example = X[i,1:ncols]
        current_label = y[i]

        if (current_label != majority_class):
            result[i]=True
            continue

        nns_labels=y[nns_indexes[i]]

        num_of_minority = 0

        for nn_label in nns_labels:
            if nn_label==i:
                continue

            if nn_label!=majority_class:
                num_of_minority = num_of_minority+1


        if num_of_minority>t:
            result[i]=False
        else:
            result[i]=True

    return(result)

In [8]:
print('Conteo inicial')
df.groupby('label').count()

Conteo inicial


,gray_matrix,hog
label,,
angry,4953,4953
disgust,547,547
fear,5121,5121
happy,8989,8989
neutral,6198,6198
sad,6077,6077
surprise,4002,4002


In [11]:
X = np.array(df['hog'].values.tolist())
y = df['label']

### Empieza la agrupacion. Pre-procesamiento.

No creo que el tiempo sea relevante. De todos modos, tengo otras operaciones en la misma computadora, lo que puede afectar a los tiempos de ejecucion (al momento de realizar este cuaderno y commitearlo el 30/10)

In [12]:
%%time
surprise=knn_und(X,y,'surprise', k=15, t=3, algorithm="auto", metric="euclidean")
surprise.count(False)

CPU times: total: 4min 3s
Wall time: 52.9 s


3396

In [13]:
%%time
angry=knn_und(X,y,'angry', k=15, t=7, algorithm="auto", metric="euclidean")
angry.count(False)

CPU times: total: 4min 9s
Wall time: 53.3 s


4390

In [14]:
%%time
fear=knn_und(X,y,'fear', k=15, t=8, algorithm="auto", metric="euclidean")
fear.count(False)

CPU times: total: 4min 14s
Wall time: 54.5 s


4386

In [15]:
%%time
sad=knn_und(X,y,'sad', k=19, t=10, algorithm="auto", metric="euclidean")
sad.count(False)

CPU times: total: 4min 21s
Wall time: 1min 1s


5491

In [16]:
%%time
neutral=knn_und(X,y,'neutral', k=15, t=5, algorithm="auto", metric="euclidean")
neutral.count(False)

CPU times: total: 4min 15s
Wall time: 58.7 s


5423

In [17]:
%%time
happy=knn_und(X,y,'happy', k=25, t=3, algorithm="auto", metric="euclidean")
happy.count(False)

CPU times: total: 4min 3s
Wall time: 1min 1s


8190

In [18]:
estados = [a*b*c*d*e*f for a,b,c,d,e,f in zip(happy,neutral,sad,fear,angry,surprise)]
df['estados']=estados
train=df[df['estados']!=0]
test=df[df['estados']!=1]

Nueva normalidad

In [21]:
train.groupby('label').count()

,gray_matrix,hog,estados
label,,,
angry,563,563,563
disgust,547,547,547
fear,735,735,735
happy,799,799,799
neutral,775,775,775
sad,586,586,586
surprise,606,606,606


In [23]:
df = df[['gray_matrix','hog' ,'estados', 'label']]
df

,gray_matrix,hog,estados,label
0,"[[46, 34, 26, 23, 12, 32, 35, 26, 26, 38, 71, ...","[0.33274356, 0.1704117, 0.025357863, 0.0006549...",0,angry
1,"[[55, 33, 54, 32, 26, 21, 50, 29, 38, 45, 79, ...","[0.27381572, 0.22986299, 0.09233569, 0.0294371...",0,angry
2,"[[123, 123, 126, 131, 124, 69, 109, 149, 159, ...","[0.34384748, 0.01883176, 0.004236705, 0.010840...",0,angry
3,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...","[0.014489936, 0.008741817, 0.020611065, 0.0, 0...",0,angry
4,"[[127, 121, 124, 137, 123, 118, 120, 111, 111,...","[0.25325525, 0.0767658, 0.026558537, 0.0181144...",0,angry
...,...,...,...,...
35882,"[[27, 26, 26, 26, 27, 27, 28, 28, 33, 34, 35, ...","[0.14022486, 0.07403437, 0.0686184, 0.0, 0.041...",0,surprise
35883,"[[7, 6, 13, 39, 76, 58, 29, 21, 16, 20, 34, 56...","[0.39263126, 0.11640715, 0.0072888294, 0.00144...",0,surprise
35884,"[[124, 129, 125, 123, 129, 128, 127, 128, 131,...","[0.20560372, 0.04748373, 0.03934206, 0.0998374...",1,surprise
35885,"[[242, 242, 242, 216, 110, 42, 20, 41, 37, 14,...","[0.3589315, 0.16316827, 0.042510495, 0.0318309...",0,surprise


Y lo empepino

In [29]:
gray_matrix = np.array(df['gray_matrix'].values)
hog = np.array(df['hog'].values)
estados = np.array(df['estados'].values)
label = np.array(df['label'].values)

np.save('gray_matrix.npy', gray_matrix)
np.save('hog.npy', hog)
np.save('estados.npy', estados)
np.save('labels.npy', label)